In [2]:
import numpy as np

import qcodes as qc
from qcodes.instrument.parameter import ManualParameter
# Classes 
from qcodes.sweep.base_sweep import (
    Nest, Zip, Chain, ParameterSweep, ParameterWrapper, FunctionSweep, FunctionWrapper
)
# Functions
from qcodes.sweep import sweep, nest, chain

In [3]:
"""
We will first make a measurement-like class which, instead of saving results to file prints them to standard output
"""

class Printer:
    def __init__(self, sweep_object): 
        self._ind, self._dep = sweep_object.parameter_table.flatten()
        self._symbols_list = sweep_object.parameter_table.symbols_list()
        
    def __enter__(self): 
        header_ind = "\t".join(["{} [{}]".format(*i) for i in self._ind.items()])
        header_dep = "\t".join(["{} [{}]".format(*i) for i in self._dep.items()])
        sep = " | "
        print((header_ind + sep + header_dep).strip(sep))
        
        return self
    
    def __exit__(self, type, value, traceback): 
        pass

    def __call__(self, result):
        print(" " + "\t ".join([str(result[ip]) for ip in self._symbols_list]))

First, let us make some parameters

In [4]:
x = ManualParameter("x", unit="V")
x(1)

y = ManualParameter("y", unit="V")
y.get = lambda: x.get() ** 2 + 1

A ParameterWrapper will make a sweep object out of a parameter. We can iterate over a sweep object a sweep object made with a ParameterWrapper will iterate once and return the get value 

In [5]:
sweep_object = ParameterWrapper(x)  # Make an iterable out of the parameter

with Printer(sweep_object) as printer:
    for i in sweep_object: # This will iterate once
        printer(i)

x [V]
 1


In [6]:
#help(ParameterWrapper)

A ParameterSweep will make a sweep object which will iterate over the array returned by the second argument and set the parameter for each value in the array sequentially. 

In [7]:
sweep_object = ParameterSweep(x, lambda: [0, 1, 2])

with Printer(sweep_object) as printer:
    for i in sweep_object: 
        printer(i)

x [V]
 0
 1
 2


A nest can be regarded as a carthesian product  $$\begin{bmatrix}x_1\\x_2\\x_3\\x_4\end{bmatrix} \times \begin{bmatrix}a()\end{bmatrix}=\begin{bmatrix}x_1a()\\x_2a()\\x_3a()\\x_4a()\end{bmatrix}$$ 

where $x_n$ is a set point n of $x$ and $a()$ represents a measurement

In [8]:
sweep_object = Nest([ParameterSweep(x, lambda: [0, 1, 2]), ParameterWrapper(y)])

with Printer(sweep_object) as printer:
    for i in sweep_object: 
        printer(i)

x [V] | y [V]
 0	 1
 1	 2
 2	 5


A 2D nest can be written as  

$$\begin{bmatrix}x_1\\x_2\\x_3\\x_4\end{bmatrix} \times \begin{bmatrix}y_1&y_2&y_3&y_4\end{bmatrix}\times\begin{bmatrix}a()\end{bmatrix}=\begin{bmatrix}x_1y_1a()&x_1y_2a()&x_1y_3a()&x_1y_4a()\\x_2y_1a()&x_2y_2a()&x_2y_3a()&x_2y_4a()\\x_3y_1a()&x_3y_2a()&x_3y_3a()&x_3y_4a()\\x_4y_1a()&x_4y_2a()&x_4y_3a()&x_4y_4a()\end{bmatrix}$$ 

where $x_n$ is a set point n of $x$ and $y_m$ represents a set point m of $y$. Also, $a()$ are measurements. The order in which the indices are returned in the loop is $x_1y_1a(), x_1y_2a(), x_1y_3a()$ etc

In [9]:
x = ManualParameter("x", unit="V")
x(1)

y = ManualParameter("y", unit="V")
y(3)

z = ManualParameter("z", unit="I")
z.get = lambda: x()**2 + 2*y()**2

sweep_object = Nest([
    ParameterSweep(x, lambda: [0, 1, 2, 4]), 
    ParameterSweep(y, lambda: [0, 1, 2, 4]), 
    ParameterWrapper(z)
])

with Printer(sweep_object) as printer:
    for i in sweep_object: 
        printer(i)

x [V]	y [V] | z [I]
 0	 0	 0
 0	 1	 2
 0	 2	 8
 0	 4	 32
 1	 0	 1
 1	 1	 3
 1	 2	 9
 1	 4	 33
 2	 0	 4
 2	 1	 6
 2	 2	 12
 2	 4	 36
 4	 0	 16
 4	 1	 18
 4	 2	 24
 4	 4	 48


With a "Chain" operator we can make a sweep as follows: $$\begin{bmatrix}x_1\\x_2\\x_3\\x_4\end{bmatrix} \times \left[\begin{bmatrix}y()\end{bmatrix}+\begin{bmatrix}z()\end{bmatrix}\right]=\begin{bmatrix}x_1y()&x_1z()\\x_2y()&x_2z()\\x_3y()&x_3z()\\x_4y()&x_4z()\end{bmatrix}$$ 

where $x_n$ is a set point n of $x$ and $y()$ represents a measurement of $y$. The order in which the elements are returned when we unroll the loop is $x_1y(), x_1z(), x_2y(), x_2z()$ etc

In [11]:
x = ManualParameter("x", unit="V")
x(1)

y = ManualParameter("y", unit="V")
y.get = lambda: x() + 3

z = ManualParameter("z", unit="I")
z.get = lambda: x()**2 + 2

sweep_object = Nest([
    ParameterSweep(x, lambda: [0, 1, 2, 3]), 
    Chain([
        ParameterWrapper(y), 
        ParameterWrapper(z) 
    ])
])

with Printer(sweep_object) as printer:
    for i in sweep_object: 
        printer(i)

x [V] | y [V]	z [I]
 0	 3	 None
 0	 None	 2
 1	 4	 None
 1	 None	 3
 2	 5	 None
 2	 None	 6
 3	 6	 None
 3	 None	 11


With a "Chain" operator we can make a sweep as follows: $$\begin{bmatrix}x_1\\x_2\\x_3\\x_4\end{bmatrix} \times \left[\begin{bmatrix}y()\end{bmatrix}+\begin{bmatrix}x_1\\x_2\\x_3\\x_4\end{bmatrix}\times\begin{bmatrix}z()\end{bmatrix}\right]=\begin{bmatrix}x_1y()&x_1z()\\x_2y()&x_2z()\\x_3y()&x_3z()\\x_4y()&x_4z()\end{bmatrix}$$ 

where $x_n$ is a set point n of $x$ and $y()$ represents a measurement of $y$. The order in which the elements are returned when we unroll the loop is $x_1y(), x_1z(), x_2y(), x_2z()$ etc

# Disallowed constructions  

The last sweep object in the nest should always be either a ParameterWrapper, or a Chain operator. Sweep objects other then the last one may *never* be a ParameterWrapper or a Chain operator